# BRO GeoTop

GeoST provides support to work with models that are provided by the [BRO](https://www.broloket.nl/ondergrondmodellen/kaart) such as GeoTOP. This section will explain the usage of [`GeoTop`](../api_reference/bro_geotop.rst), how to make easy selections and combinations with for example borehole data contained in a [`BoreholeCollection`](../api_reference/borehole_collection.rst).

## Reading GeoTop

GeoTOP data can be read from a local NetCDF file of downloaded data from the [BRO](https://www.broloket.nl/ondergrondmodellen/kaart) or directly accessed from the [OPeNDAP-server](https://dinodata.nl/opendap/GeoTOP/geotop.nc.html) of TNO Geological survey. See [`GeoTop.from_netcdf`](../api_reference/bro_geotop.rst) and [`GeoTop.from_opendap`](../api_reference/bro_geotop.rst) in the [API reference](../api_reference.md).

## Working with GeoTop

Let's load some example GeoTOP data for the Utrecht Science park:

In [ ]:
import geost

geotop = geost.data.geotop_usp()
print(geotop)

As you can see this prints information on the type of class (i.e. [`GeoTop`](../api_reference/bro_geotop.rst)), the available data variables, xyz-dimensions and resolution. The [`GeoTop`](../api_reference/bro_geotop.rst) class inherits from GeoST [`VoxelModel`](../api_reference/voxelmodel.rst) and as such, inherits all the attributes and associated selection methods. For example, we can check the horizontal and vertical bounds:

In [ ]:
print(geotop.horizontal_bounds)
print(geotop.vertical_bounds)

This prints the "xmin, ymin, xmax, ymax" bounding box and the "zmin, zmax" of the data. [`GeoTop`](../api_reference/bro_geotop.rst) depends strongly on [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) and therefore provides familiar selection methods `.sel` and `.isel` to select specific coordinates or indices, or slices of coordinates or indices:

In [ ]:
sel = geotop.sel(x=[139_650, 139_750])  # Select specific x-coordinates
sel = geotop.isel(x=[2, 5])  # Select specific x-indices

sel = geotop.sel(x=slice(140_000, 140_500))  # Select a slice of x-coordinates
sel = geotop.isel(x=slice(2, 5))  # Select a slice of x-indices

sel = geotop.sel(
    x=[140_013.23, 140_333.14], method="nearest"
)  # Select the nearest x-coordinates
print(sel)

As you can see, a selection returns a new [`GeoTop`](../api_reference/bro_geotop.rst) instance for the desired coordinates. Note that the x-resolution has also been changed because the coordinates in the last selection are approximately 300 meters apart.

### Selections

Often it can be necessary to select GeoTop at specific locations, such as borehole locations, to compare the borehole data with GeoTop. We can easily select GeoTop at the locations of boreholes using the [`GeoTop.select_with_points`](../api_reference/bro_geotop.rst) method. Let's load a [`BoreholeCollection`](../api_reference/borehole_collection.rst) with borehole data for the Utrecht Science Park to see what happens: 

In [ ]:
boreholes = geost.data.boreholes_usp()  # BoreholeCollection

geotop_at_locations = geotop.select_with_points(boreholes.header)
print(geotop_at_locations)

The result is an `xarray.Dataset` with a new "idx" dimension and note that variables like "strat" now have "(idx, z)" dimensions. The [`.select_with_points`](../api_reference/generated/geost.bro.bro_geotop.GeoTop.select_with_points.rst) method selects the voxel columns each borehole is in, in the index order of the boreholes in the [`BoreholeCollection`](../api_reference/borehole_collection.rst). It does not return a `VoxelModel` instance because a `VoxelModel` is only valid if it contains "x", "y" and "z" dimensions.

With the selection result, it is possible to compare a borehole with a specific voxel column. Let's for example compare the first borehole in the collection with the voxel column. First, let's print the "header" attribute of the borehole collection to find the "nr" of the first borehole:

In [ ]:
print(boreholes.header)

We can see that the first number (at index 0) is "B31H0541". Notice in the "surface" and "end" columns that the borehole is between +1.2 m NAP and -9.9 m NAP. We can select the first index from `geotop_at_locations`, select the depths between 1.2 and -9.9 m NAP and create a DataFrame to compare the data. In similar way, selection method for other use cases are available when working with [`GeoTop`](../api_reference/bro_geotop.rst). See the [API reference](../api_reference/bro_geotop.rst) for available methods.

In [ ]:
# Increase zmin and zmax by a little to make sure the complete depth of the borehole is covered
voxel_column = geotop_at_locations.sel(idx=0, z=slice(-10.5, 1.5))
voxel_column = voxel_column.to_dataframe().sort_index(
    ascending=False
)  # Make depth increase from top to bottom
voxel_column  # Check out the result

### Relabelling GeoTOP information

Notice that in the `voxel_column` result, the columns "strat" (i.e. stratigraphy) and "lithok" (i.e. lithology) contain numbers, instead of names which makes it difficult to understand what the unit and lithology at specific depths are. This makes a comparison with the borehole data difficult.

GeoST provides the [`StratGeotop`](../api_reference/geotop_selection.rst) class and [`Lithology`](../api_reference/geotop_selection.rst) class which make it easy to translate these numbers into more meaningful names. Let's import them and check which stratigraphic unit belongs to the number 2010 in the "strat" column and what the number 6 in the "lithok" means:

In [ ]:
from geost.bro import Lithology, StratGeotop

unit = StratGeotop.select_values(2010)
print(unit)

lith = Lithology.select_values(6)
print(lith)

As you can see, the printed result says 2010 in the "strat" column belongs to "HoloceneUnits.EC". The first part relates to the main group the unit belongs to. [`StratGeotop`](../api_reference/geotop_selection.rst) contains four main groups of stratigraphic units as class attributes: "holocene", "channel", "older" and "antropogenic". The second part is the abbreviated name of the unit which can be found in the [stratigraphic nomenclature](https://www.dinoloket.nl/stratigrafische-nomenclator/boven-noordzee-groep). Most units belong to the "Boven-Noordzee Groep" (prefix "NU"). For example, the unit "EC" from the selection result corresponds to the "Echteld formation" and can be found [here](https://www.dinoloket.nl/stratigrafische-nomenclator/formatie-van-echteld).

The printed result of `lith` says 6 in the "lithok" column refers to "Lithology.medium_sand" which is medium coarse sand.

[`StratGeotop`](../api_reference/geotop_selection.rst) and [`Lithology`](../api_reference/geotop_selection.rst) can also easily be transformed into a dictionary so each value in the "strat" and "lithok" columns can be transformed into meaningful labels:

In [ ]:
strat_dict = StratGeotop.to_dict(key="value")
lith_dict = Lithology.to_dict(key="value")

voxel_column["strat"] = voxel_column["strat"].replace(strat_dict)
voxel_column["lithok"] = voxel_column["lithok"].replace(lith_dict)
voxel_column  # See the result

Now, let's select borehole "B31H0541" from the [`BoreholeCollection`](../api_reference/borehole_collection.rst) (this was the borehole at the first index in `geotop_at_locations`) and then compare lithology and stratigraphy information in the borehole data with the voxel column.

In [ ]:
borehole = boreholes.get("B31H0541")
# Check out relevant information in the borehole data
borehole.data[["nr", "surface", "end", "top", "bottom", "lith", "strat_2003"]]

### Combining with Collections

GeoST also makes it possible to add information from GeoTop directly to borehole data in a [`BoreholeCollection`](../api_reference/borehole_collection.rst) or CPT data in a [`CptCollection`](../api_reference/cpt_collection.rst). This is shown in the [examples](../examples/combine_geotop_with_cpts.ipynb) section.